In [8]:
# -*- coding: utf-8 -*-
import os
import numpy as np
from pycode import reader,plt,process,FileChoice,testdata
import plotly
from scipy import interpolate
from scipy import signal
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.io.wavfile import read as readwav

def Csv(path):
    df = reader.CsvReader(path)
    return df["X"],df["Y"],df["S"]

def Wave(path):
    df = readwav(path)
    audio = df[1]
    return audio

def F_Fig(data,layout):
    figure = go.Figure(data,layout)
    return figure

def F_Layout(xran=None,yran=None,title=None):
    layout=go.Layout(
        xaxis=dict(range=xran),
        yaxis=dict(range=yran),
        title=title,
    )
    return layout

def F_Layout2(xran=None,yran=None,zran=None,title=None):
    layout=go.Layout(
        scene = dict(
            xaxis=dict(range=xran,title="freq"),
            yaxis=dict(range=yran,title="F.real"),
            zaxis=dict(range=zran,title="F.iamg"),
        ),
    )
    return layout

def F_Line(x,y,color=None):
    fig = go.Scatter(x=x,y=y,marker_color=color)
    return fig

def F_3DLine(x,y,z,color=None):
    fig =go.Scatter3d(
            x=x,
            y=y,
            z=z,
            showlegend=False,
            mode="lines",
            line=dict(color=color,width=2),
            )
    return fig

def F_2DHist(x,y):
    fig =go.Histogram2dContour(
            x=x,
            y=y,
            colorscale="inferno",
            showscale=False,
            reversescale=True
            )
    return fig

def F_Hist(x=None,y=None,color=None):
    fig =go.Histogram(
            x=x,
            y=y,
            marker_color=color
            )
    return fig

def F_Polar(r,theta,color):
    fig = go.Scatterpolar(
            r = r,
            theta = theta,
            mode="lines",
            line=dict(color=color,width=2),
            )
    return fig

##e.g. [[figures],[rows],[cols]] = [[figA, figB, figC],[1, 1, 1],[1, 2, 3]]
def SubFig(figs,layout,color=None,xran=None,yran=None,title=None):
    maxrows = max(figs[1])
    mincols = max(figs[2])
    fig = make_subplots(rows=maxrows,cols=maxcols)
    fig.add_trace(figs)
    layout=go.Layout(
            xaxis=dict(range=xran),
            yaxis=dict(range=yran),
            title=title,
        )
    fig.update_layout(layout)
    return fig

def Figu(t,x,color=None,xran=None,yran=None,title=None):
    fig=go.Figure(
        data=go.Scatter(x=t,y=x,marker_color=color),
        layout=go.Layout(
            xaxis=dict(range=xran),
            yaxis=dict(range=yran),
            title=title,
        )
    )
    return fig

def Figu2(x,y,xran=None,yran=None,title=None):
    fig=go.Figure(
        data=F_2DHist(x,y),
        layout=go.Layout(
            xaxis=dict(range=xran),
            yaxis=dict(range=yran),
            title=title,
        )
    )
    return fig

def Pngprint(fig,path):
    fig.write_image(path)

def HtmlExport(fig,path):
    fig.write_html(path)
    
def sep(df):
    separate=[[df[i:i+1000]] for i in range(0,6000,1000)]
    return separate

def Liner(n,t,x):
    l=interpolate.interp1d(t,x,fill_value='extrapolate')
    lx=l(n)
    return n,lx

def Hanning(n):
    hann=signal.hann(n)
    return hann

def Rectangle(n):
    rec=signal.boxcar(n)
    return rec

def RecWin(low,high,end):
    recta = np.append((Rectangle(low)-1),Rectangle(high))
    recta = np.append(recta,(Rectangle(end-high)-1))
    return recta

    
def T_Sig2amp(time,sig_x,sig_y,Sigcolor_X,Sigcolor_Y,Ampcolor_X,Ampcolor_Y,SigRan_X,SigRan_Y,AmpRan_X,AmpRan_Y,outputpath):
    dt=0.01
    sig_x -= np.mean(sig_x)
    sig_y -= np.mean(sig_y)
    
    freq,F_x_abs_amp,F_x = process.Sig2amp(time,sig_x,dt)
    freq,F_y_abs_amp,F_y = process.Sig2amp(time,sig_y,dt)
    a,b=time[0],np.ceil(time[-1])
    freq_half = max(freq)/2
    ##################
    #3D
    HtmlExport((F_Fig((F_3DLine(x=freq,y=F_x.real,z=F_x.imag,color=Ampcolor_X)),(F_Layout2(title="F.real & F.imag {} to {}".format(a,b))))),outputpath+r"\F.real_&_F.imag {} to {}.html".format(a,b))

    #polar
    sig_r,sig_phi=process.cart2pol(sig_x,sig_y)
    HtmlExport((F_Fig((F_Polar(r=sig_r,theta=sig_phi,color=Ampcolor_X)),(F_Layout2(title="pol x & pol y {} to {}".format(a,b))))),outputpath+r"\polar sig {} to {}.html".format(a,b))
    ##################


    #Signal
    Pngprint((Figu(time,sig_x,Sigcolor_X,yran=SigRan_X,title="Signal_X {} to {}".format(a,b))),outputpath+r"\Signal_X {} to {}.png".format(a,b))
    Pngprint((Figu(time,sig_y,Sigcolor_Y,yran=SigRan_Y,title="Signal_Y {} to {}".format(a,b))),outputpath+r"\Signal_Y {} to {}.png".format(a,b))

    #Amplitude
    Pngprint((F_Fig((F_Line(x=freq,y=F_x_abs_amp,color=Ampcolor_X)),(F_Layout(xran=[0,freq_half],yran=AmpRan_X,title="Amplitude_X {} to {}".format(a,b))))),outputpath+r"\Amplitude_X {} to {}.png".format(a,b))
    Pngprint((F_Fig((F_Line(x=freq,y=F_y_abs_amp,color=Ampcolor_Y)),(F_Layout(xran=[0,freq_half],yran=AmpRan_Y,title="Amplitude_Y {} to {}".format(a,b))))),outputpath+r"\Amplitude_Y {} to {}.png".format(a,b))
    
    #HeatMap
    Pngprint((Figu2(sig_x,sig_y,xran=SigRan_X,yran=SigRan_Y,title="HeatMap {} to {}".format(a,b))),outputpath+r"\HeatMap {} to {}.png".format(a,b))
    

    #Inv
    i_sig_x=process.Amp2sig(F_x)

    i_sig_y=process.Amp2sig(F_y)
    Pngprint((F_Fig((F_Line(x=time,y=i_sig_x,color=Sigcolor_X)),(F_Layout(yran=SigRan_X,title="invSignal of X {} to {}".format(a,b))))),outputpath+r"\invSignal_X {} to {}.png".format(a,b))
    Pngprint((F_Fig((F_Line(x=time,y=i_sig_y,color=Sigcolor_Y)),(F_Layout(yran=SigRan_Y,title="invSignal of Y {} to {}".format(a,b))))),outputpath+r"\invSignal_Y {} to {}.png".format(a,b))

    

    #peak
    peak_x = max(F_x_abs_amp[1:])
    peak_y = max(F_y_abs_amp[1:])

    peak_index_x = np.argmax(F_x_abs_amp[1:])+1
    peak_index_y = np.argmax(F_y_abs_amp[1:])+1
    
    peak_freq_x = freq[peak_index_x]
    peak_freq_y = freq[peak_index_y]

    #noise_rejection_Amp(cut=10freq_over)
    cut_freq = 50
    nr_F_x = np.copy(F_x)
    nr_F_y = np.copy(F_y)
    nr_F_x[(freq>cut_freq)] = 0
    nr_F_y[(freq>cut_freq)] = 0
    nr_F_x_abs_amp = np.abs(nr_F_x)/len(freq)*2
    nr_F_y_abs_amp = np.abs(nr_F_y)/len(freq)*2
    nr_F_x_abs_amp[0] = nr_F_x_abs_amp[0]/2
    nr_F_y_abs_amp[0] = nr_F_y_abs_amp[0]/2
    Pngprint((F_Fig((F_Line(x=freq,y=nr_F_x_abs_amp,color=Ampcolor_X)),(F_Layout(xran=[0,freq_half],yran=AmpRan_X,title="nr_Amplitude_X {} to {}".format(a,b))))),outputpath+r"\nr_Amplitude_X {} to {}.png".format(a,b))
    Pngprint((F_Fig((F_Line(x=freq,y=nr_F_y_abs_amp,color=Ampcolor_Y)),(F_Layout(xran=[0,freq_half],yran=AmpRan_Y,title="nr_Amplitude_Y {} to {}".format(a,b))))),outputpath+r"\nr_Amplitude_Y {} to {}.png".format(a,b))
    
    #noise_rejection_invSig(cut=10freq_over)
    nr_i_sig_x=process.Amp2sig(nr_F_x)
    nr_i_sig_y=process.Amp2sig(nr_F_y)
    nr_i_sig_x*=2
    nr_i_sig_y*=2
    Pngprint((F_Fig(data=([F_Line(x=time,y=sig_x,color=Sigcolor_X),F_Line(x=time,y=nr_i_sig_x,color=Ampcolor_Y)]),layout=(F_Layout(yran=SigRan_X,title="nr_invSignal of X {} to {}".format(a,b))))),outputpath+r"\nr_invSignal_X {} to {}.png".format(a,b))
    Pngprint((F_Fig(data=([F_Line(x=time,y=sig_y,color=Sigcolor_Y),F_Line(x=time,y=nr_i_sig_y,color=Ampcolor_X)]),layout=(F_Layout(yran=SigRan_Y,title="nr_invSignal of Y {} to {}".format(a,b))))),outputpath+r"\nr_invSignal_Y {} to {}.png".format(a,b))
    
    #noise_rejection_HeatMap
    Pngprint((Figu2(nr_i_sig_x,nr_i_sig_y,xran=SigRan_X,yran=SigRan_Y,title="nr_HeatMap {} to {}".format(a,b))),outputpath+r"\nr_HeatMap {} to {}.png".format(a,b))
    

    #peak_Amp(uncut=just peak)
    p_F_x = np.copy(F_x)
    p_F_y = np.copy(F_y)
    p_F_x[(freq!=peak_freq_x)] = 0
    p_F_y[(freq!=peak_freq_y)] = 0
    p_F_x_abs_amp = np.abs(p_F_x)/len(freq)*2
    p_F_y_abs_amp = np.abs(p_F_y)/len(freq)*2
    p_F_x_abs_amp[0] = p_F_x_abs_amp[0]/2
    p_F_y_abs_amp[0] = p_F_y_abs_amp[0]/2
    Pngprint((F_Fig(data=([F_Line(x=freq,y=F_x_abs_amp,color=Ampcolor_X),F_Line(x=freq,y=p_F_x_abs_amp,color=Sigcolor_X)]),layout=(F_Layout(xran=[0,freq_half],yran=AmpRan_X,title="peak_Amplitude_X {} to {}".format(a,b))))),outputpath+r"\p_Amplitude_X {} to {}.png".format(a,b))
    Pngprint((F_Fig(data=([F_Line(x=freq,y=F_y_abs_amp,color=Ampcolor_Y),F_Line(x=freq,y=p_F_y_abs_amp,color=Sigcolor_Y)]),layout=(F_Layout(xran=[0,freq_half],yran=AmpRan_Y,title="peak_Amplitude_Y {} to {}".format(a,b))))),outputpath+r"\p_Amplitude_Y {} to {}.png".format(a,b))
    
    #peak_invSig(cut=just peak)
    p_i_sig_x=process.Amp2sig(p_F_x)
    p_i_sig_y=process.Amp2sig(p_F_y)
    p_i_sig_x*=2
    p_i_sig_y*=2
    Pngprint((F_Fig(data=([F_Line(x=time,y=sig_x,color=Sigcolor_X),F_Line(x=time,y=p_i_sig_x,color=Ampcolor_Y)]),layout=(F_Layout(yran=SigRan_X,title="peak_invSignal of X {} to {}".format(a,b))))),outputpath+r"\peak_invSignal_X {} to {}.png".format(a,b))
    Pngprint((F_Fig(data=([F_Line(x=time,y=sig_y,color=Sigcolor_Y),F_Line(x=time,y=p_i_sig_y,color=Ampcolor_X)]),layout=(F_Layout(yran=SigRan_Y,title="peak_invSignal of Y {} to {}".format(a,b))))),outputpath+r"\peak_invSignal_Y {} to {}.png".format(a,b))
    
    #scatter
    Pngprint((F_Fig(data=([F_Line(x=p_i_sig_x,y=p_i_sig_y,color="black")]),layout=(F_Layout(xran=SigRan_X,yran=SigRan_Y,title="peak_Scatter {} to {}".format(a,b))))),outputpath+r"\peak_scatter {} to {}.png".format(a,b))

    #noise_rejection_HeatMap
    Pngprint((Figu2(p_i_sig_x,p_i_sig_y,xran=SigRan_X,yran=SigRan_Y,title="peak_HeatMap {} to {}".format(a,b))),outputpath+r"\peak_HeatMap {} to {}.png".format(a,b))
    
    return (peak_x,peak_y,peak_freq_x,peak_freq_y)

In [6]:
def Test(inputpath,outputpath,Sigcolor_X,Ampcolor_X,Sigcolor_Y,Ampcolor_Y,SigRan_X,AmpRan_X,SigRan_Y,AmpRan_Y):
    #__init__
    if not os.path.exists(outputpath):
        os.mkdir(outputpath)
    print("{} is converting...".format(outputpath))

    peak_X,peak_Y=[],[]
    peak_index_X=[]
    peak_index_Y=[]
    peak_freq_X,peak_freq_Y=[],[]
    samp=["0 to 10","10 to 20","20 to 30","30 to 40","40 to 50","50 to 60"]
    rec=Rectangle(6001)
    N = np.linspace(0,60,num=6001)
    r_sig_x,r_sig_y,r_time = Csv(inputpath)

    #### Process ###
    time,sig_x = Liner(N,r_time,r_sig_x)
    time,sig_y = Liner(N,r_time,r_sig_y)

    time,sig_x = time*rec,sig_x*rec
    time,sig_y = time*rec,sig_y*rec

    #ALL
    T_Sig2amp(time,sig_x,sig_y,Sigcolor_X,Sigcolor_Y,Ampcolor_X,Ampcolor_Y,SigRan_X,SigRan_Y,AmpRan_X,AmpRan_Y,outputpath)
    #Sepalation
    sep_time = sep(time)
    sep_sig_x = sep(sig_x)
    sep_sig_y = sep(sig_y)

    for i in range(len(sep_time)):
        p=T_Sig2amp(sep_time[i][0],sep_sig_x[i][0],sep_sig_y[i][0],Sigcolor_X,Sigcolor_Y,Ampcolor_X,Ampcolor_Y,SigRan_X,SigRan_Y,AmpRan_X,AmpRan_Y,outputpath)
        peak_X += [p[0]]
        peak_Y += [p[1]]
        peak_freq_X += [p[2]]
        peak_freq_Y += [p[3]]
###
    Pngprint((Figu(samp,peak_X,"black",yran=[0,10],title="peak_X")),outputpath+"\peak_X.png")
    Pngprint((Figu(samp,peak_freq_X,"black",yran=None,title="peak_freq_X")),outputpath+r"\peak_freq_X.png")

    Pngprint((Figu(samp,peak_Y,"black",yran=[0,10],title="peak_Y")),outputpath+"\peak_Y.png")
    Pngprint((Figu(samp,peak_freq_Y,"black",yran=None,title="peak_freq_Y")),outputpath+r"\peak_freq_Y.png")
    
    print(" Done.")
    print("")

def FourierTest(inputpath,outputpath):
    Test(
    inputpath = inputpath,
    outputpath = outputpath,

    ### Parameter ###
    Sigcolor_X="red",
    Ampcolor_X="orangered",
    Sigcolor_Y="blue",
    Ampcolor_Y="cyan",

    #range
    SigRan_X=[-30,30],
    AmpRan_X=[0,10],
    SigRan_Y=[-30,30],
    AmpRan_Y=[0,10]
    )

In [9]:
def Paramater():
    datasets=[
        #testdata.d_190714OCData(),
        testdata.d_honda(),
        testdata.d_hukuma(),
        testdata.d_ikeda(),
        testdata.d_miyamoto(),
        testdata.d_morita(),
        testdata.d_nawate(),
        testdata.d_ohno(),
        testdata.d_orimoto(),
        testdata.d_printer(),
    ]
    for dataset in datasets:
        inputpath = dataset
        """
        #printer
        r'.\data\printer\Printer200523200224.csv',
        r'.\data\printer\Printer200523200802.csv',
        #honda
        r'.\data\honda\BC00.csv',
        r'.\data\honda\BO00.csv',
        r'.\data\honda\TC00.csv',
        r'.\data\honda\TO00.csv',
        #nawate
        r'.\data\nawate\BC00.csv',
        r'.\data\nawate\BO00.csv',
        r'.\data\nawate\TC00.csv',
        r'.\data\nawate\TO00.csv',
        """
        base = [os.path.splitext(inputpath[_]) for _ in range(len(inputpath))]
        outputpath=[base[j][0] for j in range(len(base))]

        for i in range(len(outputpath)):
            print("No.{}".format(i+1))
            FourierTest(inputpath[i],outputpath[i])
    print("All Success.")

Paramater()

No.1
C:/Users/b215029/Desktop/CoG/data/Honda/BC00 is converting...
 Done.

No.2
C:/Users/b215029/Desktop/CoG/data/Honda/BO00 is converting...
 Done.

No.3
C:/Users/b215029/Desktop/CoG/data/Honda/TC00 is converting...
 Done.

No.4
C:/Users/b215029/Desktop/CoG/data/Honda/TO00 is converting...
 Done.

No.1
C:/Users/b215029/Desktop/CoG/data/Hukuma/BC_hukuma01190528172156 is converting...
 Done.

No.2
C:/Users/b215029/Desktop/CoG/data/Hukuma/BC_hukuma02190528172329 is converting...
 Done.

No.3
C:/Users/b215029/Desktop/CoG/data/Hukuma/BM_hukuma05190528172846 is converting...
 Done.

No.4
C:/Users/b215029/Desktop/CoG/data/Hukuma/BO_hukuma03190528172515 is converting...
 Done.

No.5
C:/Users/b215029/Desktop/CoG/data/Hukuma/BO_hukuma04190528172639 is converting...
 Done.

No.6
C:/Users/b215029/Desktop/CoG/data/Hukuma/TC_hukuma06190528173032 is converting...
 Done.

No.7
C:/Users/b215029/Desktop/CoG/data/Hukuma/TC_hukuma08190528173412 is converting...
 Done.

No.8
C:/Users/b215029/Desktop/CoG/d

In [ ]:
honda = testdata.d_honda()
print(honda)

In [ ]:
def RecWin(low,high,end):
    recta = np.append((Rectangle(low)-1),Rectangle(high))
    recta = np.append(recta,(Rectangle(end)-1))
    return recta
RecWin(0,5,10)

In [ ]:
"""""""""#FileChoice.App()

    #### Window 10sec ###
    def Sep10s(sep_time,sep_sig_x,sep_sig_y)
        for i in range(len(sn)):
            dt=0.001
            freq,amp_x,F_x = process.Sig2amp(time,sig_x,dt)
            freq,amp_y,F_y = process.Sig2amp(time,sig_y,dt)

            msx = max(fsx)
            msy = max(fsy)

            peak_X += [msx]
            peak_index_X += [np.argmax(fsx)]
            peak_Y += [msy]
            peak_index_Y += [np.argmax(fsy)]

            Pngprint((Figu(nsn,nsx,Sigcolor_X,yran=SigRan_X,title="Sample Signal_X {} to {}".format(i*10,(i+1)*10))),outputpath+r"\Sample Signal_X {} to {}.png".format(i*10,(i+1)*10))
            
            Pngprint((Figu(fsn,fsx,Ampcolor_X,yran=AmpRan_X,title="Sample Amplitude_X {} to {}".format(i*10,(i+1)*10))),outputpath+r"\Sample Amplitude_X .png".format(i*10,(i+1)*10))
            
            Pngprint((Figu(nsn,nsy,Sigcolor_Y,yran=SigRan_Y,title="Sample Signal_Y {} to {}".format(i*10,(i+1)*10))),outputpath+r"\Sample Signal_Y {} to {}.png".format(i*10,(i+1)*10))

            Pngprint((Figu(fsn,fsy,Ampcolor_Y,yran=AmpRan_Y,title="Sample Amplitude_Y {} to {}".format(i*10,(i+1)*10))),outputpath+r"\Sample Amplitude_Y {} to {}.png".format(i*10,(i+1)*10))
            
            Pngprint((Figu2(nsx,nsy,xran=SigRan_X,yran=SigRan_Y,title="Sample HeatMap {} to {}".format(i*10,(i+1)*10))),outputpath+r"\Sample all HeatMap {} to {}.png".format(i*10,(i+1)*10))


"""""""""